# Proyecto 03 - Procesamiento del Lenguaje Natural

## Dataset: The Multilingual Amazon Reviews Corpus

**Recuerda descargar el dataset de [aquí](https://github.com/kang205/SASRec). Es un archivo .zip que contiene tres documentos. Más información sobre el dataset [aquí](https://registry.opendata.aws/amazon-reviews-ml/). Es importante que tengas en cuenta la [licencia](https://docs.opendata.aws/amazon-reviews-ml/license.txt) de este dataset.**

### Exploración de datos y Procesamiento del Lenguaje Natural

Dedícale un buen tiempo a hacer un Análisis Exploratorio de Datos. Considera que hasta que no hayas aplicado las herramientas de Procesamiento del Lenguaje Natural vistas, será difícil completar este análisis. Elige preguntas que creas que puedas responder con este dataset. Por ejemplo, ¿qué palabras están asociadas a calificaciones positivas y qué palabras a calificaciones negativas?

### Machine Learning

Implementa un modelo que, dada la crítica de un producto, asigne la cantidad de estrellas correspondiente. **Para pensar**: ¿es un problema de Clasificación o de Regresión?

1. Haz todas las transformaciones de datos que consideres necesarias. Justifica.
1. Evalúa de forma apropiada sus resultados. Justifica la métrica elegida.
1. Elige un modelo benchmark y compara tus resultados con este modelo.
1. Optimiza los hiperparámetros de tu modelo.
1. Intenta responder la pregunta: ¿Qué información está usando el modelo para predecir?

**Recomendación:** si no te resulta conveniente trabajar en español con NLTK, te recomendamos que explores la librería [spaCy](https://spacy.io/).

### Para pensar, investigar y, opcionalmente, implementar
1. ¿Valdrá la pena convertir el problema de Machine Learning en un problema binario? Es decir, asignar únicamente las etiquetas Positiva y Negativa a cada crítica y hacer un modelo que, en lugar de predecir las estrellas, prediga esa etiqueta. Pensar en qué situación puede ser útil. ¿Esperas que el desempeño sea mejor o peor?
1. ¿Hay algo que te gustaría investigar o probar?

### **¡Tómate tiempo para investigar y leer mucho!**

In [1]:
import pandas as pd
import seaborn as sns
import nltk
import matplotlib.pyplot as plt

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

import es_core_news_sm

from collections import Counter
import itertools
from wordcloud import WordCloud

In [ ]:
df = pd.read_json('dataset_es_dev.json',lines=True)
df_test = pd.read_json('dataset_es_test.json',lines=True)
df_train = pd.read_json('dataset_es_train.json',lines=True)

# A - Exploración de datos y Procesamiento del Lenguaje Natural

Tanto el dataset de formación como el de prueba cuentan con <code>5000 reviews</code>, mientras que el de entrenamiento posee <code>200.000 instancias</code>

In [ ]:
print(df.shape)
print(df_test.shape)
print(df_train.shape)

Todos cuentan con 7 atributos:
- ID de reseña
- ID de producto
- ID de evaluador
- Estrellas
- Cuerpo de reseña
- Título de reseña
- Idioma
- Categoría de producto

Ninguno de los cuales posee valores faltantes

In [ ]:
print(df.info())
print(df_test.info())
print(df_train.info())

La totalidad de las reseñas se encuentran en idioma español

In [ ]:
print(df['language'].unique())
print(df_test['language'].unique())
print(df_train['language'].unique())

Dado que para explorar el dataset completo resulta difícil trabajar con los 3 por separado, los concatenaré a continuación

In [ ]:
df_amazon = pd.concat([df,df_test,df_train],axis=0)
df_amazon.head(5)

Puedo observar del siguiente gráfico que cada puntaje posee la misma cantidad de reseñas asociadas

In [ ]:
sns.countplot(x = 'stars', data = df_amazon)
plt.ylabel('Cantidad de reseñas')
plt.xlabel('Puntaje (estrellas)')

plt.show()

In [ ]:
print('Existen',str(len(df_amazon['product_category'].unique())),'categorías únicas en el presente dataset')

Podemos observar que predominan las reseñas de productos línea **hogar** e **inalámbricos**

In [ ]:
plt.figure(figsize = (10,7))

sns.countplot(y = 'product_category', 
              data = df_amazon,  
              order = df_amazon.product_category.value_counts().index)
plt.ylabel('Categoría',size=13)
plt.xlabel('Cantidad de reviews')
plt.tick_params(axis='y', labelsize=13)
plt.title('Categorías de los diferentes productos', size=25)
plt.show()

## Preprocesamiento de las reseñas

In [ ]:
df_amazon['review_entero'] = df_amazon['review_body']+ ' ' + df_amazon['review_title']
df_amazon.reset_index(inplace=True,drop=True)
df_amazon.head(5)

Cargo las palabras comunes (stopwords) en español para luego filtrar las distintas 

In [ ]:
# Función que aplica procesamiento del lenguaje natural a palabras en español
nlp = es_core_news_sm.load()

stop_words = nlp.Defaults.stop_words

A través de la siguiente función se aplica <code>**normalizado**</code> al texto de cada fila. Esto incluye:
- <code>Tokenizado</code> (separación de una secuencia de palabras en partes, almacenadas en una lista)
- Todas las palabras pasan a ser minúsculas
- Se eliminan comas, puntos y palabras de 1 letra (por ejemplo "y", "a", etc)
- Se quitan las <code>stopwords</code>. Estas son palabras que carecen de significado por sí solas, como preposiciones, artículos, conjunciones, pronombres, etc.
- <code>Stemming</code> y <code>lemmatizado</code>. En el **stemming** se recorta el principio o el final de la palabra que suele ser común a las diferentes conjugaciones (por ejemplo, palabras "niñez", o "niñas" al ser stemmizadas quedarían "niñ"), mientras que el **lemmatizado** tiene en cuenta el análisis morfológico de la palabra (por ejemplo, palabras como "estudiando" o "estudios" serían transformadas a "estudio")

El objetivo de este preprocesamiento, que reducirá el ruido del texto al eliminar las formas infleccionales y a veces derivacionalmente relacionadas a una base común*, es facilitar el posterior análisis de cada reseña mediante algoritmos de clasificación. De esta forma, se podrán predecir con mayor precisión la cantidad de estrellas que daría un usuario dada una reseña

*"**Stemming and lemmatization", Stanford, Cambridge University Press, 2008, https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html**


McNary, Dave. “Keanu Reeves, Alex Winter Returning for ‘Bill and Ted Face the Music.’” Variety, Penske Media Corporation, 8 May 2018, variety.com/2018/film/news/bill-and-ted-3-keanu-reeves-alex-winter-1202802946/.

In [ ]:
def procesar(texto):
    reseña_tk = nltk.RegexpTokenizer('\w+').tokenize(texto)
    reseña_tk_minusculas = [word.lower() for word in reseña_tk if word.lower() not in stop_words]
    reseña_limpia = ' '.join(reseña_tk_minusculas)
    reseña_limpia = nlp(reseña_limpia)
    reseña_lemm = [word.lemma_ for word in reseña_limpia]
    reseña_lemm = [t for t in reseña_lemm if len(t)>1]
    return reseña_lemm

In [ ]:
procesar(df_amazon.review_entero[0])

In [ ]:
#def procesar(texto):
#    palabras = []
#    doc = nlp(texto)
#    for sent in doc.sentences:
#        for word in sent.words:
#            palabras.append(word.lemma)
#    palabras = list(filter(lambda x: x != "," and x != ".", palabras))
#    palabras = [item for item in palabras if item not in stopwords]
#    return palabras
#procesar(df_amazon['review_entero'][1])

In [ ]:
df_amazon.head(5)

In [ ]:
df_amazon['review_entero_tk'] = df_amazon.apply(lambda row: procesar(row['review_entero']), axis=1)

In [ ]:
df_amazon.to_excel('Preprocesado.xlsx')

In [2]:
df_amazon = pd.read_excel('Preprocesado.xlsx').drop(columns='Unnamed: 0')

In [1]:
df_amazon.head(5)

NameError: name 'df_amazon' is not defined

In [ ]:
for x in range(1,6):
    tokenizado_df = df_amazon[df_amazon['stars']==x]
    tokenizado_lista = tokenizado_df.review_entero_tk.tolist()

    lista = [x for l in tokenizado_lista for x in l]

    frecuencia_palabras = Counter(lista)
    palabras_comunes = frecuencia_palabras.most_common()
    
    print(f'5 palabras más comunes para críticas de {x} estrellas')
    df_frec = pd.DataFrame(palabras_comunes, columns = ['Palabras', 'Frecuencia'])
    print(df_frec.head(5))
    print('-------------')
    
    fig,(ax1,ax2) = plt.subplots(2,1,figsize=(800,400))
    ax1 = plt.subplot(211)
    u_string=(" ").join(lista)
    wc = WordCloud(max_words = 50,width=800,height=400).generate(u_string)
    plt.imshow(wc)
    plt.axis("off")
    plt.title(f'Nube de palabras en críticas de {x} estrellas', fontsize=100)

    
    ax2 = sns.barplot(ax=ax2,x = df_frec.iloc[:20].Palabras, y = df_frec.iloc[:20].Frecuencia)
    ax2.set(xlabel='Frecuencias', ylabel='Palabras')
    ax2.set_title(f'Frecuencia de palabras en críticas de {x} estrellas')
    plt.show()
    print('-------------')

5 palabras más comunes para críticas de 1 estrellas
  Palabras  Frecuencia
0        '     1033814
1        r      477573
2        ,      474913
3               474913
4        a      437064
-------------
